<a href="https://colab.research.google.com/github/OptionalLanguor/datasci4health/blob/dev/notebooks/e3_per_recording_bandpower_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Compute the average bandpower of an EEG signal
---

One of the most widely used method to analyze EEG data is to decompose the signal into functionally distinct frequency bands, such as delta (0.5–4 Hz), theta (4–8 Hz), alpha (8–12 Hz), beta (12–30 Hz), and gamma (30–100 Hz).

This implies the decomposition of the EEG signal into frequency components, which is commonly achieved through Fourier transforms. The almost invariably used algorithm to compute the Fourier transform (and arguably the most important signal processing algorithm) is the Fast Fourier Transform (FFT), which returns, for each frequency bin, a complex number from which one can then easily extract the amplitude and phase of the signal at that specific frequency. In spectral analysis, it is then common to take the magnitude-squared of the FFT to obtain an estimate of the power spectral density (or power spectrum, or periodogram), expressed in (micro)-Volts2 per Hertz in the case of EEG data.

Although a myriad of analyses can be performed from the power spectral density, I am going to focus here on a very simple one: the average band power, which consists in computing a single number that summarizes the contribution of the given frequency band to the overall power of the signal. This could be particularly useful in a machine-learning approach, when often you will want to extract some keys features from your data (and have a single number that could summarize a particular aspect of your data).

In order to compute the average bandpower in the delta band, we first need to compute an estimate of the power spectral density. The most widely-used method to do that is the Welch's periodogram, which consists in averaging consecutive Fourier transform of small windows of the signal, with or without overlapping.

The Welch's method improves the accuracy of the classic periodogram. The reason is simple: EEG data are always time-varying, meaning that if you look at a 30 seconds of EEG data, it is very (very) unlikely that the signal will looks like a perfect sum of pure sines. Rather, the spectral content of the EEG changes over time, constantly modified by the neuronal activity at play under the scalp. Problem is, to return a true spectral estimate, a classic periodogram requires the spectral content of the signal to be stationnary (i.e. time-unvarying) over the time period considered. Because it is never the case, the periodogram is generally biased and contains way too much variance (see the end of this tutorial). By averaging the periodograms obtained over short segments of the windows, the Welch's method allows to drastically reduce this variance. This comes at the cost, however, of a lower frequency resolution. Indeed, the frequency resolution is defined by:

$Fres = \frac{Fs}{N} = \frac{Fs}{Fst} = \frac{1}{t}$

where: 
* Fs is the sampling frequency of the signal
* N is the total number of samples
* t is the duration in seconds of the signal. 

How do we define the optimal window duration then? A commonly used approach is to take a window sufficiently long to encompasses at least two full cycles of the lowest frequency of interest.


In [ ]:
# load participant metadata
import pandas as pd

commom_path = "https://raw.githubusercontent.com/OptionalLanguor/datasci4health/master/data/external/eeglab_dataset/"
participants_file = "participants.tsv"

url_participants = commom_path + participants_file
participants = pd.read_csv(url_participants, sep="\t")
participants.head()

,participant_id,Original_ID,Group,sess1_Med,sess2_Med,sex,age
0,sub-001,8010,CTL,NaN,no s2,Female,61
1,sub-002,801,PD,ON,OFF,Female,60
2,sub-003,802,PD,OFF,ON,Male,75
3,sub-004,803,PD,OFF,ON,Female,76
4,sub-005,804,PD,ON,OFF,Male,75


In [ ]:
def bandpower(data, sf, band, window_sec=None, relative=False):
    """Compute the average power of the signal x in a specific frequency band.

    Parameters
    ----------
    data : 1d-array
        Input signal in the time-domain.
    sf : float
        Sampling frequency of the data.
    band : list
        Lower and upper frequencies of the band of interest.
    window_sec : float
        Length of each window in seconds.
        If None, window_sec = (1 / min(band)) * 2
    relative : boolean
        If True, return the relative power (= divided by the total power of the signal).
        If False (default), return the absolute power.

    Return
    ------
    bp : float
        Absolute or relative band power.
    """
    from scipy.signal import welch
    from scipy.integrate import simps
    import numpy as np
    band = np.asarray(band)
    low, high = band

    # Define window length
    if window_sec is not None:
        nperseg = window_sec * sf
    else:
        nperseg = (2 / low) * sf

    # Compute the modified periodogram (Welch)
    freqs, psd = welch(data, sf, nperseg=nperseg)

    # Frequency resolution
    freq_res = freqs[1] - freqs[0]

    # Find closest indices of band in frequency vector
    idx_band = np.logical_and(freqs >= low, freqs <= high)

    # Integral approximation of the spectrum using Simpson's rule.
    bp = simps(psd[idx_band], dx=freq_res)

    if relative:
        bp /= simps(psd, dx=freq_res)
    #else:
     #   WelchPeriodogram(freqs,psd,band)
    return bp

In [ ]:
def WelchPeriodogram(freqs, psd, band):
    """Plot Welch's Periodogram

    Parameters
    ----------
    freqs : 1d-array
        Input signal in the time-domain.
    psd : float
        Sampling frequency of the data.

    Return
    ------
    None
    """
    import matplotlib.pyplot as plt
    import seaborn as sns
    import numpy as np

    # Define freq lower and upper limits
    low, high = band

    # Find intersecting values in frequency vector
    idx_freq = np.logical_and(freqs >= low, freqs <= high)

    # Plot the power spectrum
    sns.set(font_scale=1.2)
    plt.figure(figsize=(8, 4))
    plt.plot(freqs, psd, color='k', lw=2)
    plt.fill_between(freqs, psd, where=idx_freq, color='skyblue')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Power spectral density (V^2 / Hz)')
    plt.ylim([0, psd.max() * 1.1])
    plt.title("Welch's periodogram")
    plt.xlim([0, freqs.max()])
    plt.show()

In [ ]:
from google.colab import drive
!pip install mne

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from glob import glob
import matplotlib.pyplot as plt
import mne
import numpy as np
import seaborn as sns
sns.set(font_scale=1.2)
import pandas as pd
import itertools

set_files_paths = glob('gdrive/MyDrive/Projeto Final/EEG_Dataset_reupload/sub-*/ses-*/eeg/sub-*_ses-*_task-SimonConflict_eeg.set')
len(set_files_paths)

# directory for output of ".csv" files with descritive statistics
out_path = "gdrive/MyDrive/bandpower/"
bp_df = pd.DataFrame(columns=['absolute_bp','relative_bp','freq_act','sub','ses','channel'])
channels_TBT = ['Cz','Fz','Pz','Oz','X','Y','Z']

for i, each in enumerate(set_files_paths):
  print(f"Subject file ({i+1}/{len(set_files_paths)}): {each}\n")
  f_raw = mne.io.read_raw_eeglab(each)
 
  print("Performing BandPower Analysis")
  data = f_raw.to_data_frame()
  data = data.drop(columns=['time']) #drop non electrode column

  #for electrode in data.columns:
  for electrode in channels_TBT:

    data = f_raw.to_data_frame()
    data = data.drop(columns=['time']) #drop non electrode column
  
    # Droppin all channels but the current one
    data = data.filter([electrode])
    data = np.asarray(data).reshape(1,-1)[0]

    # Define sampling frequency and time vector
    sf = 500
    time = np.arange(data.size) / sf

    # Plot the signal
    #fig, ax = plt.subplots(1, 1, figsize=(12, 4))
    #plt.plot(time, data, lw=1.5, color='k')
    #plt.xlabel('Time (seconds)')
    #plt.ylabel('Voltage')
    #plt.xlim([time.min(), time.max()])
    #plt.title(f'EEG data - Cz Electrode Subject {i+1}')
    #plt.show()

    # Frequency activities
    freq_act = {'delta':(.5,4),'tetha':(4,8),'alpha':(8,12),'beta':(12,30),'gama':(30,100)}

    for freq in freq_act:
      # Compute the absolute power by approximating the area under the curve
      beta_power = bandpower(data, sf, freq_act[freq])
      print('Absolute beta power: %.3f uV^2' % beta_power)

      # Relative beta power (expressed as a percentage of total power)
      beta_rel_power = bandpower(data, sf, freq_act[freq], relative=True)
      print('Relative beta power: %.3f' % beta_rel_power)

      print(f'Frequency: {freq}, Sub: {each[73:76]}, Ses: {each[81:83]}, Channel: {electrode}')

      # Save .csv
      temp_df = pd.DataFrame(
          {
          'absolute_bp': [beta_power],
          'relative_bp': [beta_rel_power],
          'freq_act': [freq],
          'sub': [each[73:76]],
          'ses': [each[81:83]],
          'channel': [electrode],
          })
      bp_df = pd.concat([bp_df,temp_df], ignore_index=True)
      df_bandpower_filepath = out_path + 'Cz_Fz_Pz_Oz_bandPowers.csv'
      bp_df.to_csv(df_bandpower_filepath)

Subject file (1/84): gdrive/MyDrive/Projeto Final/EEG_Dataset_reupload/sub-001/ses-01/eeg/sub-001_ses-01_task-SimonConflict_eeg.set

Reading /content/gdrive/MyDrive/Projeto Final/EEG_Dataset_reupload/sub-001/ses-01/eeg/sub-001_ses-01_task-SimonConflict_eeg.fdt


<ipython-input-10-9025a06b9f66>:20: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  f_raw = mne.io.read_raw_eeglab(each)


Performing BandPower Analysis
Absolute beta power: 7.482 uV^2
Relative beta power: 0.245
Frequency: delta, Sub: 001, Ses: 01, Channel: Cz
Absolute beta power: 1.626 uV^2
Relative beta power: 0.139
Frequency: tetha, Sub: 001, Ses: 01, Channel: Cz
Absolute beta power: 0.999 uV^2
Relative beta power: 0.104
Frequency: alpha, Sub: 001, Ses: 01, Channel: Cz
Absolute beta power: 1.868 uV^2
Relative beta power: 0.230
Frequency: beta, Sub: 001, Ses: 01, Channel: Cz
Absolute beta power: 1.967 uV^2
Relative beta power: 0.315
Frequency: gama, Sub: 001, Ses: 01, Channel: Cz
Absolute beta power: 177.699 uV^2
Relative beta power: 0.322
Frequency: delta, Sub: 001, Ses: 01, Channel: Fz
Absolute beta power: 41.122 uV^2
Relative beta power: 0.229
Frequency: tetha, Sub: 001, Ses: 01, Channel: Fz
Absolute beta power: 14.386 uV^2
Relative beta power: 0.117
Frequency: alpha, Sub: 001, Ses: 01, Channel: Fz
Absolute beta power: 9.974 uV^2
Relative beta power: 0.123
Frequency: beta, Sub: 001, Ses: 01, Channel: 

<ipython-input-10-9025a06b9f66>:20: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  f_raw = mne.io.read_raw_eeglab(each)


Performing BandPower Analysis
Absolute beta power: 22.694 uV^2
Relative beta power: 0.162
Frequency: delta, Sub: 003, Ses: 01, Channel: Cz
Absolute beta power: 8.198 uV^2
Relative beta power: 0.246
Frequency: tetha, Sub: 003, Ses: 01, Channel: Cz
Absolute beta power: 4.478 uV^2
Relative beta power: 0.158
Frequency: alpha, Sub: 003, Ses: 01, Channel: Cz
Absolute beta power: 5.350 uV^2
Relative beta power: 0.222
Frequency: beta, Sub: 003, Ses: 01, Channel: Cz
Absolute beta power: 4.680 uV^2
Relative beta power: 0.322
Frequency: gama, Sub: 003, Ses: 01, Channel: Cz
Absolute beta power: 136.495 uV^2
Relative beta power: 0.411
Frequency: delta, Sub: 003, Ses: 01, Channel: Fz
Absolute beta power: 40.419 uV^2
Relative beta power: 0.266
Frequency: tetha, Sub: 003, Ses: 01, Channel: Fz
Absolute beta power: 17.428 uV^2
Relative beta power: 0.155
Frequency: alpha, Sub: 003, Ses: 01, Channel: Fz
Absolute beta power: 14.688 uV^2
Relative beta power: 0.176
Frequency: beta, Sub: 003, Ses: 01, Channel

...

Performing BandPower Analysis
Absolute beta power: 51370823.297 uV^2
Relative beta power: 0.331
Frequency: delta, Sub: 026, Ses: 02, Channel: Cz
Absolute beta power: 1889527.676 uV^2
Relative beta power: 0.094
Frequency: tetha, Sub: 026, Ses: 02, Channel: Cz
Absolute beta power: 435954.654 uV^2
Relative beta power: 0.059
Frequency: alpha, Sub: 026, Ses: 02, Channel: Cz
Absolute beta power: 193000.376 uV^2
Relative beta power: 0.066
Frequency: beta, Sub: 026, Ses: 02, Channel: Cz
Absolute beta power: 113435.147 uV^2
Relative beta power: 0.119
Frequency: gama, Sub: 026, Ses: 02, Channel: Cz
Absolute beta power: 5695.991 uV^2
Relative beta power: 0.390
Frequency: delta, Sub: 026, Ses: 02, Channel: Fz
Absolute beta power: 372.707 uV^2
Relative beta power: 0.158
Frequency: tetha, Sub: 026, Ses: 02, Channel: Fz
Absolute beta power: 126.327 uV^2
Relative beta power: 0.107
Frequency: alpha, Sub: 026, Ses: 02, Channel: Fz
Absolute beta power: 90.288 uV^2
Relative beta power: 0.135
Frequency: be

<ipython-input-10-9025a06b9f66>:20: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  f_raw = mne.io.read_raw_eeglab(each)


Performing BandPower Analysis
Absolute beta power: 3.152 uV^2
Relative beta power: 0.216
Frequency: delta, Sub: 026, Ses: 01, Channel: Cz
Absolute beta power: 1.088 uV^2
Relative beta power: 0.173
Frequency: tetha, Sub: 026, Ses: 01, Channel: Cz
Absolute beta power: 0.545 uV^2
Relative beta power: 0.096
Frequency: alpha, Sub: 026, Ses: 01, Channel: Cz
Absolute beta power: 0.776 uV^2
Relative beta power: 0.168
Frequency: beta, Sub: 026, Ses: 01, Channel: Cz
Absolute beta power: 1.338 uV^2
Relative beta power: 0.399
Frequency: gama, Sub: 026, Ses: 01, Channel: Cz
Absolute beta power: 70.270 uV^2
Relative beta power: 0.503
Frequency: delta, Sub: 026, Ses: 01, Channel: Fz
Absolute beta power: 10.081 uV^2
Relative beta power: 0.167
Frequency: tetha, Sub: 026, Ses: 01, Channel: Fz
Absolute beta power: 4.109 uV^2
Relative beta power: 0.105
Frequency: alpha, Sub: 026, Ses: 01, Channel: Fz
Absolute beta power: 5.194 uV^2
Relative beta power: 0.196
Frequency: beta, Sub: 026, Ses: 01, Channel: Fz

<ipython-input-10-9025a06b9f66>:20: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  f_raw = mne.io.read_raw_eeglab(each)


Performing BandPower Analysis
Absolute beta power: 20.995 uV^2
Relative beta power: 0.402
Frequency: delta, Sub: 027, Ses: 02, Channel: Cz
Absolute beta power: 2.776 uV^2
Relative beta power: 0.138
Frequency: tetha, Sub: 027, Ses: 02, Channel: Cz
Absolute beta power: 1.384 uV^2
Relative beta power: 0.104
Frequency: alpha, Sub: 027, Ses: 02, Channel: Cz
Absolute beta power: 2.349 uV^2
Relative beta power: 0.235
Frequency: beta, Sub: 027, Ses: 02, Channel: Cz
Absolute beta power: 2.019 uV^2
Relative beta power: 0.299
Frequency: gama, Sub: 027, Ses: 02, Channel: Cz
Absolute beta power: 387.898 uV^2
Relative beta power: 0.717
Frequency: delta, Sub: 027, Ses: 02, Channel: Fz
Absolute beta power: 29.887 uV^2
Relative beta power: 0.120
Frequency: tetha, Sub: 027, Ses: 02, Channel: Fz
Absolute beta power: 8.068 uV^2
Relative beta power: 0.069
Frequency: alpha, Sub: 027, Ses: 02, Channel: Fz
Absolute beta power: 7.009 uV^2
Relative beta power: 0.119
Frequency: beta, Sub: 027, Ses: 02, Channel: 